# 다산콜센터 질의-응답 데이터 EDA

## 데이터셋 개요
- **출처**: AI Hub - 민원(콜센터) 질의-응답 데이터
- **도메인**: 다산콜센터 (서울시 120 콜센터)
- **데이터 형태**: 질의-응답 대화쌍 + 음성 데이터
- **카테고리**: 
  - 대중교통 안내
  - 코로나19 관련 상담
  - 일반행정 문의
  - 생활하수도 관련 문의

---

## 1. Setup & 한국어 폰트 설정

In [ ]:
import os
import json
from pathlib import Path
from collections import Counter, defaultdict
from typing import Dict, List, Any

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
from tqdm.notebook import tqdm

# Pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)

# Visualization settings
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
%matplotlib inline

In [ ]:
# 한국어 폰트 설정 (macOS/Windows/Linux 호환)
def setup_korean_font():
    """
    한국어 폰트를 자동으로 찾아서 설정합니다.
    macOS, Windows, Linux에서 모두 작동합니다.
    """
    # 시스템별 한국어 폰트 후보
    font_candidates = [
        # macOS
        'AppleGothic', 'Apple SD Gothic Neo', 'AppleMyungjo',
        # Windows
        'Malgun Gothic', 'Gulim', 'Batang', 'Dotum',
        # Linux
        'NanumGothic', 'NanumBarunGothic', 'UnDotum',
        # 기타
        'DejaVu Sans'
    ]
    
    # 사용 가능한 폰트 찾기
    available_fonts = [f.name for f in fm.fontManager.ttflist]
    
    for font_name in font_candidates:
        if font_name in available_fonts:
            plt.rcParams['font.family'] = font_name
            plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지
            print(f"✓ 한국어 폰트 설정: {font_name}")
            return font_name
    
    # 폰트를 찾지 못한 경우
    print("⚠️  한국어 폰트를 찾을 수 없습니다. 기본 폰트를 사용합니다.")
    print("   나눔고딕 설치 권장: https://hangeul.naver.com/2017/nanum")
    return None

# 폰트 설정 실행
korean_font = setup_korean_font()
print(f"\n사용 가능한 한국어 폰트 목록 (처음 10개):")
korean_fonts = [f.name for f in fm.fontManager.ttflist if 'Korean' in f.name or 'Hangul' in f.name or 'Nanum' in f.name]
for font in korean_fonts[:10]:
    print(f"  - {font}")

## 2. 데이터 경로 설정

In [ ]:
# Base paths
BASE_DIR = Path("/Users/sdh/Dev/02_production_projects/humetro-ai-assistant")
DATA_DIR = BASE_DIR / "data/dasan_call/extracted"
OUTPUT_DIR = BASE_DIR / "data/processed/dasan_eda"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Data paths
TRAINING_LABELED = DATA_DIR / "training/labeled"
TRAINING_SOURCE = DATA_DIR / "training/source"
VALIDATION_LABELED = DATA_DIR / "validation/labeled"
VALIDATION_SOURCE = DATA_DIR / "validation/source"

print(f"Data Directory: {DATA_DIR}")
print(f"Output Directory: {OUTPUT_DIR}")
print(f"\nData exists: {DATA_DIR.exists()}")

## 3. 데이터 로드 함수

In [ ]:
def load_json_files(directory: Path) -> pd.DataFrame:
    """
    디렉토리에서 모든 JSON 파일을 로드하여 DataFrame으로 반환
    
    Args:
        directory: JSON 파일이 있는 디렉토리 경로
    
    Returns:
        통합된 DataFrame
    """
    all_data = []
    json_files = list(directory.glob("*.json"))
    
    print(f"Loading {len(json_files)} JSON files from {directory.name}...")
    
    for json_file in tqdm(json_files, desc="Loading files"):
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                
                # 파일명에서 카테고리 추출
                filename = json_file.stem
                
                # DataFrame 생성
                if isinstance(data, list):
                    df = pd.DataFrame(data)
                    df['source_file'] = filename
                    all_data.append(df)
                    
        except Exception as e:
            print(f"Error loading {json_file.name}: {e}")
    
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        print(f"✓ Loaded {len(combined_df):,} records")
        return combined_df
    else:
        print("⚠️  No data loaded")
        return pd.DataFrame()

## 4. Training 데이터 로드

In [ ]:
# Load training data
print("="*60)
print("LOADING TRAINING DATA")
print("="*60)
training_df = load_json_files(TRAINING_LABELED)

print(f"\nTraining data shape: {training_df.shape}")
print(f"Columns: {list(training_df.columns)}")
training_df.head()

## 5. Validation 데이터 로드

In [ ]:
# Load validation data
print("="*60)
print("LOADING VALIDATION DATA")
print("="*60)
validation_df = load_json_files(VALIDATION_LABELED)

print(f"\nValidation data shape: {validation_df.shape}")
validation_df.head()

## 6. 데이터 통합

In [ ]:
# Add split column
training_df['split'] = 'Training'
validation_df['split'] = 'Validation'

# Combine datasets
full_df = pd.concat([training_df, validation_df], ignore_index=True)

print("="*60)
print("DATASET OVERVIEW")
print("="*60)
print(f"Total records: {len(full_df):,}")
print(f"Training: {len(training_df):,} ({len(training_df)/len(full_df)*100:.1f}%)")
print(f"Validation: {len(validation_df):,} ({len(validation_df)/len(full_df)*100:.1f}%)")
print()
print("Data types:")
print(full_df.dtypes)

## 7. 기본 통계

In [ ]:
# 도메인 분포
print("="*60)
print("도메인 분포")
print("="*60)
print(full_df['도메인'].value_counts())
print()

# 카테고리 분포
print("="*60)
print("카테고리 분포")
print("="*60)
print(full_df['카테고리'].value_counts())
print()

# 화자 분포
print("="*60)
print("화자 분포")
print("="*60)
print(full_df['화자'].value_counts())

## 8. 카테고리별 분석

In [ ]:
# 카테고리별 통계
category_stats = full_df.groupby(['카테고리', 'split']).size().unstack(fill_value=0)
category_stats['Total'] = category_stats.sum(axis=1)
category_stats = category_stats.sort_values('Total', ascending=False)

print("="*60)
print("카테고리별 레코드 수")
print("="*60)
print(category_stats)
print()

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 카테고리별 분포
category_counts = full_df['카테고리'].value_counts()
axes[0].barh(range(len(category_counts)), category_counts.values)
axes[0].set_yticks(range(len(category_counts)))
axes[0].set_yticklabels(category_counts.index)
axes[0].set_xlabel('레코드 수')
axes[0].set_title('카테고리별 데이터 분포')
axes[0].grid(axis='x', alpha=0.3)

# Train/Validation 분포
split_counts = full_df['split'].value_counts()
axes[1].pie(split_counts.values, labels=split_counts.index, autopct='%1.1f%%',
           startangle=90, colors=['#3498db', '#e74c3c'])
axes[1].set_title('Train vs Validation 분포')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'category_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ 시각화 저장: {OUTPUT_DIR / 'category_distribution.png'}")

## 9. 대화쌍 분석

In [ ]:
# 대화셋 일련번호별 통계
dialogue_stats = full_df.groupby('대화셋일련번호').agg({
    '문장번호': 'count',
    '카테고리': 'first'
}).rename(columns={'문장번호': 'num_turns'})

print("="*60)
print("대화쌍 통계")
print("="*60)
print(f"총 대화 수: {len(dialogue_stats):,}")
print(f"평균 턴 수: {dialogue_stats['num_turns'].mean():.2f}")
print(f"최대 턴 수: {dialogue_stats['num_turns'].max()}")
print(f"최소 턴 수: {dialogue_stats['num_turns'].min()}")
print()
print("턴 수 분포:")
print(dialogue_stats['num_turns'].describe())

# 턴 수 분포 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 히스토그램
axes[0].hist(dialogue_stats['num_turns'], bins=30, edgecolor='black', alpha=0.7)
axes[0].axvline(dialogue_stats['num_turns'].mean(), color='red', linestyle='--',
               label=f'평균: {dialogue_stats["num_turns"].mean():.1f}')
axes[0].set_xlabel('대화 턴 수')
axes[0].set_ylabel('빈도')
axes[0].set_title('대화당 턴 수 분포')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# 박스플롯 (카테고리별)
category_turns = full_df.groupby(['카테고리', '대화셋일련번호']).size().reset_index(name='turns')
category_turns.boxplot(column='turns', by='카테고리', ax=axes[1])
axes[1].set_xlabel('카테고리')
axes[1].set_ylabel('턴 수')
axes[1].set_title('카테고리별 대화 턴 수 분포')
plt.setp(axes[1].xaxis.get_majorticklabels(), rotation=45, ha='right')
plt.suptitle('')  # Remove default title

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'dialogue_turns_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ 시각화 저장: {OUTPUT_DIR / 'dialogue_turns_analysis.png'}")

## 10. QA 구조 분석

In [ ]:
# QA 분포
print("="*60)
print("QA 구조 분석")
print("="*60)
qa_dist = full_df['QA'].value_counts()
print(qa_dist)
print()

# 화자별 QA 분포
speaker_qa = pd.crosstab(full_df['화자'], full_df['QA'])
print("화자별 QA 분포:")
print(speaker_qa)

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# QA 분포 파이차트
axes[0].pie(qa_dist.values, labels=qa_dist.index, autopct='%1.1f%%',
           startangle=90)
axes[0].set_title('질의(Q) vs 응답(A) 분포')

# 화자별 QA 히트맵
sns.heatmap(speaker_qa, annot=True, fmt='d', cmap='YlOrRd', ax=axes[1],
           cbar_kws={'label': '레코드 수'})
axes[1].set_title('화자별 QA 분포')
axes[1].set_xlabel('QA')
axes[1].set_ylabel('화자')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'qa_structure_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ 시각화 저장: {OUTPUT_DIR / 'qa_structure_analysis.png'}")

## 11. 의도 분석

In [ ]:
# 고객 의도 분석
print("="*60)
print("고객 의도 TOP 20")
print("="*60)
customer_intent = full_df['고객의도'].value_counts().head(20)
print(customer_intent)
print()

# 상담사 의도 분석
print("="*60)
print("상담사 의도 TOP 20")
print("="*60)
agent_intent = full_df['상담사의도'].value_counts().head(20)
print(agent_intent)

# 시각화
fig, axes = plt.subplots(2, 1, figsize=(14, 12))

# 고객 의도 TOP 15
top_customer = customer_intent.head(15)
axes[0].barh(range(len(top_customer)), top_customer.values)
axes[0].set_yticks(range(len(top_customer)))
axes[0].set_yticklabels(top_customer.index)
axes[0].set_xlabel('빈도')
axes[0].set_title('고객 의도 TOP 15')
axes[0].grid(axis='x', alpha=0.3)
axes[0].invert_yaxis()

# 상담사 의도 TOP 15
top_agent = agent_intent.head(15)
axes[1].barh(range(len(top_agent)), top_agent.values, color='coral')
axes[1].set_yticks(range(len(top_agent)))
axes[1].set_yticklabels(top_agent.index)
axes[1].set_xlabel('빈도')
axes[1].set_title('상담사 의도 TOP 15')
axes[1].grid(axis='x', alpha=0.3)
axes[1].invert_yaxis()

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'intent_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ 시각화 저장: {OUTPUT_DIR / 'intent_analysis.png'}")

## 12. 텍스트 길이 분석

In [ ]:
# 고객 질문/응답 길이
full_df['customer_q_len'] = full_df['고객질문(요청)'].fillna('').str.len()
full_df['customer_a_len'] = full_df['고객답변'].fillna('').str.len()

# 상담사 질문/응답 길이
full_df['agent_q_len'] = full_df['상담사질문(요청)'].fillna('').str.len()
full_df['agent_a_len'] = full_df['상담사답변'].fillna('').str.len()

print("="*60)
print("텍스트 길이 통계")
print("="*60)
length_stats = full_df[['customer_q_len', 'customer_a_len', 'agent_q_len', 'agent_a_len']].describe()
print(length_stats)

# 시각화
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 고객 질문 길이
axes[0, 0].hist(full_df[full_df['customer_q_len'] > 0]['customer_q_len'],
               bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('길이 (글자 수)')
axes[0, 0].set_ylabel('빈도')
axes[0, 0].set_title('고객 질문 길이 분포')
axes[0, 0].grid(axis='y', alpha=0.3)

# 고객 응답 길이
axes[0, 1].hist(full_df[full_df['customer_a_len'] > 0]['customer_a_len'],
               bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[0, 1].set_xlabel('길이 (글자 수)')
axes[0, 1].set_ylabel('빈도')
axes[0, 1].set_title('고객 응답 길이 분포')
axes[0, 1].grid(axis='y', alpha=0.3)

# 상담사 질문 길이
axes[1, 0].hist(full_df[full_df['agent_q_len'] > 0]['agent_q_len'],
               bins=50, edgecolor='black', alpha=0.7, color='lightgreen')
axes[1, 0].set_xlabel('길이 (글자 수)')
axes[1, 0].set_ylabel('빈도')
axes[1, 0].set_title('상담사 질문 길이 분포')
axes[1, 0].grid(axis='y', alpha=0.3)

# 상담사 응답 길이
axes[1, 1].hist(full_df[full_df['agent_a_len'] > 0]['agent_a_len'],
               bins=50, edgecolor='black', alpha=0.7, color='lightskyblue')
axes[1, 1].set_xlabel('길이 (글자 수)')
axes[1, 1].set_ylabel('빈도')
axes[1, 1].set_title('상담사 응답 길이 분포')
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'text_length_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ 시각화 저장: {OUTPUT_DIR / 'text_length_analysis.png'}")

## 13. 개체명/용어사전/지식베이스 분석

In [ ]:
# 개체명 분석
print("="*60)
print("개체명 분석")
print("="*60)

# 개체명이 있는 레코드 비율
entity_ratio = (full_df['개체명 '].notna() & (full_df['개체명 '] != '')).sum() / len(full_df) * 100
print(f"개체명이 있는 레코드: {entity_ratio:.2f}%")
print()

# 용어사전이 있는 레코드 비율
term_ratio = (full_df['용어사전'].notna() & (full_df['용어사전'] != '')).sum() / len(full_df) * 100
print(f"용어사전이 있는 레코드: {term_ratio:.2f}%")
print()

# 지식베이스가 있는 레코드 비율
kb_ratio = (full_df['지식베이스'].notna() & (full_df['지식베이스'] != '')).sum() / len(full_df) * 100
print(f"지식베이스가 있는 레코드: {kb_ratio:.2f}%")

# 시각화
fig, ax = plt.subplots(figsize=(10, 6))

categories = ['개체명', '용어사전', '지식베이스']
ratios = [entity_ratio, term_ratio, kb_ratio]

bars = ax.bar(categories, ratios, color=['#3498db', '#e74c3c', '#2ecc71'])
ax.set_ylabel('비율 (%)')
ax.set_title('개체명/용어사전/지식베이스 존재 비율')
ax.set_ylim(0, 100)
ax.grid(axis='y', alpha=0.3)

# 값 표시
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
           f'{height:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'annotation_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ 시각화 저장: {OUTPUT_DIR / 'annotation_analysis.png'}")

## 14. 샘플 데이터 확인

In [ ]:
# 각 카테고리별 샘플 대화 1개씩 출력
print("="*60)
print("카테고리별 샘플 대화")
print("="*60)

for category in full_df['카테고리'].unique():
    print(f"\n{'='*60}")
    print(f"카테고리: {category}")
    print("="*60)
    
    # 해당 카테고리의 첫 번째 대화 선택
    category_data = full_df[full_df['카테고리'] == category]
    first_dialogue_id = category_data['대화셋일련번호'].iloc[0]
    dialogue = category_data[category_data['대화셋일련번호'] == first_dialogue_id].sort_values('문장번호')
    
    print(f"대화 ID: {first_dialogue_id}")
    print(f"턴 수: {len(dialogue)}\n")
    
    for _, row in dialogue.head(5).iterrows():
        speaker = "고객" if row['화자'] == '고객' else "상담사"
        qa = row['QA']
        
        if qa == 'Q':
            text = row['고객질문(요청)'] if speaker == "고객" else row['상담사질문(요청)']
        else:
            text = row['고객답변'] if speaker == "고객" else row['상담사답변']
        
        if pd.notna(text) and text:
            print(f"[{speaker}] {text}")
    
    if len(dialogue) > 5:
        print(f"... (총 {len(dialogue)}턴)")

## 15. 데이터 품질 체크

In [ ]:
print("="*60)
print("데이터 품질 체크")
print("="*60)

# 결측값 확인
print("\n결측값:")
missing = full_df.isnull().sum()
missing_pct = (missing / len(full_df) * 100).round(2)
missing_df = pd.DataFrame({'Count': missing, 'Percentage': missing_pct})
print(missing_df[missing_df['Count'] > 0])

# 중복 대화셋 확인
duplicate_dialogues = full_df['대화셋일련번호'].duplicated().sum()
print(f"\n중복 문장 (같은 대화셋 내): {duplicate_dialogues:,}")

# 화자별 레코드 수
print("\n화자별 레코드 수:")
print(full_df['화자'].value_counts())

# 빈 텍스트 확인
empty_customer_q = (full_df['고객질문(요청)'].isna() | (full_df['고객질문(요청)'] == '')).sum()
empty_customer_a = (full_df['고객답변'].isna() | (full_df['고객답변'] == '')).sum()
empty_agent_q = (full_df['상담사질문(요청)'].isna() | (full_df['상담사질문(요청)'] == '')).sum()
empty_agent_a = (full_df['상담사답변'].isna() | (full_df['상담사답변'] == '')).sum()

print(f"\n빈 텍스트:")
print(f"  고객 질문: {empty_customer_q:,} ({empty_customer_q/len(full_df)*100:.1f}%)")
print(f"  고객 응답: {empty_customer_a:,} ({empty_customer_a/len(full_df)*100:.1f}%)")
print(f"  상담사 질문: {empty_agent_q:,} ({empty_agent_q/len(full_df)*100:.1f}%)")
print(f"  상담사 응답: {empty_agent_a:,} ({empty_agent_a/len(full_df)*100:.1f}%)")

## 16. 요약 통계 저장

In [ ]:
# 요약 통계 생성
summary_stats = {
    'total_records': len(full_df),
    'total_dialogues': full_df['대화셋일련번호'].nunique(),
    'training_records': len(training_df),
    'validation_records': len(validation_df),
    'categories': full_df['카테고리'].unique().tolist(),
    'avg_dialogue_turns': dialogue_stats['num_turns'].mean(),
    'entity_coverage_pct': entity_ratio,
    'term_coverage_pct': term_ratio,
    'kb_coverage_pct': kb_ratio,
    'category_distribution': full_df['카테고리'].value_counts().to_dict(),
    'speaker_distribution': full_df['화자'].value_counts().to_dict(),
    'qa_distribution': full_df['QA'].value_counts().to_dict()
}

# JSON으로 저장
with open(OUTPUT_DIR / 'summary_statistics.json', 'w', encoding='utf-8') as f:
    json.dump(summary_stats, f, indent=2, ensure_ascii=False)

print("="*60)
print("요약 통계")
print("="*60)
for key, value in summary_stats.items():
    if isinstance(value, float):
        print(f"{key}: {value:.2f}")
    elif isinstance(value, dict):
        print(f"{key}: {len(value)} items")
    elif isinstance(value, list):
        print(f"{key}: {value}")
    else:
        print(f"{key}: {value:,}" if isinstance(value, int) else f"{key}: {value}")

print(f"\n✓ 요약 통계 저장: {OUTPUT_DIR / 'summary_statistics.json'}")

## 17. CSV 저장

In [ ]:
# Full dataset을 CSV로 저장
output_csv = OUTPUT_DIR / 'dasan_full_dataset.csv'
full_df.to_csv(output_csv, index=False, encoding='utf-8-sig')
print(f"✓ 전체 데이터셋 저장: {output_csv}")
print(f"  크기: {output_csv.stat().st_size / (1024*1024):.2f} MB")

# 카테고리별 요약
category_summary = full_df.groupby('카테고리').agg({
    '대화셋일련번호': 'nunique',
    '문장번호': 'count',
    'split': lambda x: f"{(x=='Training').sum()}/{(x=='Validation').sum()}"
}).rename(columns={
    '대화셋일련번호': '대화 수',
    '문장번호': '전체 레코드',
    'split': 'Train/Val'
})

category_summary.to_csv(OUTPUT_DIR / 'category_summary.csv', encoding='utf-8-sig')
print(f"✓ 카테고리 요약 저장: {OUTPUT_DIR / 'category_summary.csv'}")

print("\n" + "="*60)
print("카테고리별 요약")
print("="*60)
print(category_summary)

## 18. 결론 및 다음 단계

### 주요 발견사항:
- 총 레코드 수와 대화 수 확인
- 카테고리별 데이터 분포 균형 확인
- 대화 턴 수 및 텍스트 길이 패턴 파악
- 개체명/용어사전/지식베이스 커버리지 분석

### 데이터 품질:
- 결측값 및 빈 텍스트 비율
- 화자별/QA별 분포 균형
- 의도 레이블 다양성

### 다음 단계:
1. **텍스트 전처리**: 개인정보 마스킹 검증, 특수문자 처리
2. **토큰화 및 어휘 분석**: 형태소 분석, 주요 키워드 추출
3. **임베딩 생성**: 문장 임베딩, 의도 임베딩 생성
4. **모델 학습 준비**: Train/Val 분리, 배치 생성
5. **평가 메트릭 정의**: 의도 분류, 응답 생성 평가 지표

In [ ]:
print("="*60)
print("✅ EDA 완료")
print("="*60)
print(f"\n모든 결과가 저장되었습니다: {OUTPUT_DIR}")
print("\n생성된 파일:")
for file in sorted(OUTPUT_DIR.glob('*')):
    print(f"  - {file.name}")